In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import cyipopt
from pybullet_suite import *

pybullet build time: May 20 2022 19:44:17


In [2]:
num_steps = 5
p_obs = jnp.array([0, 0.1])
r_obs = 0.2

dim_robot = 7
r_robot = 0.1
safe_dist = 0.05

In [3]:
bw = BulletWorld(gui=True)
sm = BulletSceneMaker(bw)

argv[0]=
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=3
argv[0] = --unused
argv[1] = 
argv[2] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3090/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 515.105.01
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 515.105.01
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3090/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation


In [4]:
pandas = [bw.load_robot(Panda, f'panda{i}') for i in range(5)]

In [5]:
panda = pandas[0]
q_init = jnp.array([0.,0.,0.,-1.5708,0.,1.8675, 0.])
q_goal = panda.get_random_config()

In [6]:
# util functions
def to_mat(x):
    return x.reshape(-1, dim_robot)
def at_timestep(i, x):
    return to_mat(x)[i]
def to_v(x):
    return (to_mat(x)[1:] - to_mat(x)[:-1]).flatten()

In [7]:
# functions
def min_dist_cost(x):
    v = to_v(x)
    return v @ v

In [8]:
from functools import partial
state_init = partial(at_timestep, 0)
state_goal = partial(at_timestep,-1)

In [9]:
g, cl, cu = [], [], []
f = min_dist_cost
g.append(state_init)
cl.extend(q_init)
cu.extend(q_init)
g.append(state_goal)
cl.extend(q_goal)
cu.extend(q_goal)

In [10]:
lb = jnp.hstack([panda.joint_lower_limit] * num_steps)
ub = jnp.hstack([panda.joint_upper_limit] * num_steps)
cl = jnp.array(cl)
cu = jnp.array(cu)

dim = dim_robot*num_steps
#const_dim = len(cl)
#

In [11]:
from nlp_solver import NLPSolver

In [12]:
solver = NLPSolver(dim)
solver.build(f, g)

In [23]:
x0 = jnp.hstack([panda.joint_central]*num_steps)
xsol, info = solver.solve(x0, lb, ub, cl, cu, {'tol':0.001})

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       14
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       63

Total number of variables............................:       35
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       35
                     variables with only upper bounds:        0
Total number of equality constraints.................:       14
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.41e+00 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [21]:
for i in range(num_steps):
    pandas[i].set_joint_angles(to_mat(xsol)[i])